## The notebook working appropriate with jupyter notebook!
##### For using in google colab exchange the markdown cells to code

In [1]:
import json
import os
from pony.orm import *
import pandas as pd
import random
import urllib.request
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import operator

# Part 1- setting kaggle's datasets API

##### Google colab:

api_token = {"username":"ENTER_YOUR_USERNAME","key":"ENTER_YOUR_KEY"}
with open('./kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [2]:
#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"yossiabu","key":"df75a3186740c346f9de87cbed4f6d00"}
kaggle_json_path = r'C:/Users/yossi/.kaggle/kaggle.json'
# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('./kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)
os.chmod(kaggle_json_path, mode=600)

In [3]:
# searching for the dataset
!kaggle datasets list -s baby

ref                                           title                              size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------  --------------------------------  -----  -------------------  -------------  ---------  ---------------  
kaggle/us-baby-names                          US Baby Names                     173MB  2017-11-21 22:18:15          31120        324  0.5882353        
thedevastator/us-baby-names-by-year-of-birth  Baby Names by Year                  9MB  2022-09-20 06:26:33            797         37  1.0              
ironicninja/baby-names                        Baby Names                          5MB  2021-02-09 16:21:40            467         12  0.88235295       
haezer/french-baby-names                      French Baby Names                  18MB  2020-04-07 16:32:38            437         15  0.88235295       
robikscube/us-baby-name-popularity            US Baby Name Popularity            52MB  2

### Download baby dataset

In [4]:
os.mkdir('./datasets/')
os.mkdir('./datasets/us-baby-name/')
#download the dataset from Kaggle and unzip it
!kaggle datasets download kaggle/us-baby-names -f database.sqlite -p ./datasets/us-baby-name/database.sqlite
!chdir ./datasets/us-baby-name/database.sqlite/
!unzip ./datasets/us-baby-name/database.sqlite/*.zip  -d ./datasets/us-baby-name/database.sqlite


  0%|          | 0.00/131M [00:00<?, ?B/s]
  1%|          | 1.00M/131M [00:00<01:06, 2.05MB/s]
  2%|2         | 3.00M/131M [00:00<00:23, 5.72MB/s]
  4%|3         | 5.00M/131M [00:00<00:17, 7.66MB/s]
  5%|5         | 7.00M/131M [00:01<00:15, 8.43MB/s]
  8%|7         | 10.0M/131M [00:01<00:10, 12.4MB/s]
  9%|9         | 12.0M/131M [00:01<00:09, 13.7MB/s]
 11%|#         | 14.0M/131M [00:01<00:09, 12.5MB/s]
 14%|#3        | 18.0M/131M [00:01<00:06, 18.3MB/s]
 16%|#6        | 21.0M/131M [00:01<00:05, 19.3MB/s]
 18%|#8        | 24.0M/131M [00:01<00:05, 20.1MB/s]
 21%|##        | 27.0M/131M [00:02<00:05, 21.2MB/s]
 23%|##2       | 30.0M/131M [00:02<00:04, 22.4MB/s]
 25%|##5       | 33.0M/131M [00:02<00:04, 22.7MB/s]
 27%|##7       | 36.0M/131M [00:02<00:04, 20.9MB/s]
 30%|###       | 40.0M/131M [00:02<00:04, 23.4MB/s]
 33%|###2      | 43.0M/131M [00:02<00:03, 23.6MB/s]
 35%|###5      | 46.0M/131M [00:02<00:03, 23.6MB/s]
 37%|###7      | 49.0M/131M [00:03<00:03, 22.7MB/s]
 40%|###9      | 52.

 71%|#######   | 93.0M/131M [00:05<00:01, 20.8MB/s]
 73%|#######3  | 96.0M/131M [00:05<00:02, 17.8MB/s]
 75%|#######5  | 99.0M/131M [00:05<00:01, 19.3MB/s]
 78%|#######7  | 102M/131M [00:05<00:01, 20.5MB/s] 
 80%|########  | 105M/131M [00:05<00:01, 21.7MB/s]
 82%|########2 | 108M/131M [00:06<00:01, 21.2MB/s]
 85%|########4 | 111M/131M [00:06<00:00, 22.5MB/s]
 87%|########6 | 114M/131M [00:06<00:00, 21.7MB/s]
 89%|########9 | 117M/131M [00:06<00:00, 17.8MB/s]
 91%|######### | 119M/131M [00:06<00:00, 15.8MB/s]
 93%|#########2| 122M/131M [00:06<00:00, 17.8MB/s]
 95%|#########5| 125M/131M [00:07<00:00, 19.5MB/s]
 98%|#########7| 128M/131M [00:07<00:00, 20.9MB/s]
100%|#########9| 131M/131M [00:07<00:00, 21.8MB/s]
100%|##########| 131M/131M [00:07<00:00, 18.7MB/s]


Archive:  ./datasets/us-baby-name/database.sqlite/database.sqlite.zip
  inflating: ./datasets/us-baby-name/database.sqlite/database.sqlite  


##### Google colab:

#Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/us-baby-name

#download the dataset from Kaggle and unzip it
!kaggle datasets download kaggle/us-baby-names -f database.sqlite -p ./datasets/us-baby-name/database.sqlite
!chdir ./datasets/us-baby-name/database.sqlite/
!unzip ./datasets/us-baby-name/database.sqlite/*.zip  -d ./datasets/us-baby-name/database.sqlite

# Part 2- working with SQLite

In [5]:
DB_PATH = './datasets/us-baby-name/database.sqlite'

**Question 1: How many names in the dataset?**

To answer this question, we need to run a query on the database. This can be done by doing following actions:

1.   We connect to the database, which is a local file, using the *connect* function.
2.   We create a *cursor object*.
3.   We call the *execute function* to run a query
4.   We fetch the results using the *fetchone* function


In [6]:
conn = sqlite3.connect(f'{DB_PATH}/database.sqlite') # connecting to the database
c = conn.cursor() # creating a cursor object

In [7]:
national_q = "SELECT count(*) FROM NationalNames"
state_q = "SELECT count(*) FROM StateNames"
print("The number of national names in the dataset: %s" % c.execute(national_q ).fetchone()) # execute a query & fetch the results
print("The number of state names in the dataset: %s" % c.execute(state_q ).fetchone()) 

The number of national names in the dataset: 1825433
The number of state names in the dataset: 5647426


**Question 2: How many distinct names in the Washington State? and in New York State?**

We can answer this question, by using the [Select Distinct] statment(https://www.w3schools.com/sql/sql_distinct.asp):

In [8]:
washington_q = \
'''
SELECT count(DISTINCT Name) 
FROM StateNames
WHERE State = 'WA'
'''

In [9]:
ny_q = \
'''
SELECT count(DISTINCT Name) 
FROM StateNames
WHERE State = 'NY'
'''

In [10]:
print("The number of distinct names in the Washington State in the dataset: %s" % c.execute(washington_q).fetchone()) # execute a query & fetch the results
print("The number of distinct names in the New York State in the dataset: %s" % c.execute(ny_q ).fetchone()) 

The number of distinct names in the Washington State in the dataset: 5828
The number of distinct names in the New York State in the dataset: 13933


**Question 3: What are the most rare names?**

To answer this question, we need to count the number of times each name appeared in the dataset. This can be done by using the following steps: First, we will use the [Group By](https://www.w3schools.com/sql/sql_groupby.asp) statement, and SQL [Sum function](https://www.w3schools.com/sql/sql_count_avg_sum.asp). Second, we will use the [Order By](https://www.w3schools.com/sql/sql_orderby.asp) keyword to order the results. Lastly, we iterate through the retrieved results using a for loop: 

In [11]:
rare_q = \
'''
SELECT Name, Sum(Count) as TotalNumberOfBabies
             FROM NationalNames
             Group By Name
             Order by Sum(Count)
'''

In [12]:
rows = c.execute(rare_q) 
i = 1 
for r in rows:
    if i > 20:
        break
    print(f"{i}. {r[0]}, {r[1]}")
    #print({0}, {1}, {2}.format(i, r[0], r[1]))
    i += 1

1. Aabid, 5
2. Aadhyan, 5
3. Aadian, 5
4. Aadrian, 5
5. Aadrit, 5
6. Aafreen, 5
7. Aagot, 5
8. Aahron, 5
9. Aaiyana, 5
10. Aaja, 5
11. Aakanksha, 5
12. Aakira, 5
13. Aakiyah, 5
14. Aaleeya, 5
15. Aalias, 5
16. Aalicia, 5
17. Aaliyaa, 5
18. Aallyah, 5
19. Aalon, 5
20. Aalyia, 5


Now let's print the names in descending order:

**Question 4: What are the most common names?**

In [13]:
rare_q_desc = \
'''
SELECT Name, Sum(Count) as TotalNumberOfBabies
             FROM NationalNames
             Group By Name
             Order by Sum(Count) DESC
'''

In [14]:
rows = c.execute(rare_q_desc) 
i = 1 
for r in rows:
    if i > 20:
        break
    print(f"{i}. {r[0]}, {r[1]}")
    #print({0}, {1}, {2}.format(i, r[0], r[1]))
    i += 1

1. James, 5129096
2. John, 5106590
3. Robert, 4816785
4. Michael, 4330805
5. Mary, 4130441
6. William, 4071368
7. David, 3590557
8. Joseph, 2580687
9. Richard, 2564867
10. Charles, 2376700
11. Thomas, 2291517
12. Christopher, 2004177
13. Daniel, 1876880
14. Elizabeth, 1606282
15. Patricia, 1575529
16. Matthew, 1558671
17. Jennifer, 1467573
18. George, 1464430
19. Linda, 1454599
20. Barbara, 1437083


**Question 5:** Write a code that queries the babies DB you prepared and retrieves the number of male babies named George in 1992.

In [15]:
george_q = \
'''
SELECT count
FROM NationalNames
WHERE Name = 'George'
AND Gender = 'M'
AND Year = '1992'
'''
result = c.execute(george_q).fetchone()[0]
print('Number of male babies named George in 1992:', result)

Number of male babies named George in 1992: 3892


***Question 6:*** Write a function that returns how many babies were born in a given state in a given year.
Use it to calculate the number of babies born in TX in 1986.

In [16]:
def get_babies_count(state, year):
    q = \
    '''
    SELECT sum(count)
    FROM StateNames
    WHERE State = '{0}'
    AND Year = '{1}'
    '''.format(state, year)
    return 'Number of babies in state {0} in year {1}: {2}'.format(state, year, c.execute(q).fetchone()[0])

In [17]:
get_babies_count('TX', '1986')

'Number of babies in state TX in year 1986: 277779'

**Question 7:** Write a function that returns how many male babies named Robert where born in a given state in a given year. 
Use it to find the state in which the highest number of babies Robert were born in 2010 (10pt)

In [18]:
def get_robert_count(state, year):
    q = \
    '''
    SELECT count
    FROM StateNames
    WHERE State = '{0}'
    AND Year = '{1}'
    AND Name = 'Robert'
    AND Gender = 'M'
    '''.format(state, year)
    return c.execute(q).fetchone()[0]

In [19]:
uniqueStates_q = \
'''
SELECT DISTINCT(State)
FROM StateNames
'''
allStatesTuple = c.execute(uniqueStates_q).fetchall()

In [20]:
allStatesList = [temp[0] for temp in allStatesTuple]

In [21]:
allRoberetCount = [get_robert_count(temp, '2010') for temp in allStatesList]

In [22]:
robertCountList = list(zip(allStatesList, allRoberetCount))

In [23]:
robertCountList.sort(key=operator.itemgetter(1), reverse=True)

In [24]:
robertCountList

[('CA', 852),
 ('TX', 688),
 ('NY', 504),
 ('FL', 447),
 ('PA', 343),
 ('GA', 302),
 ('OH', 297),
 ('IL', 270),
 ('NJ', 252),
 ('NC', 240),
 ('MI', 226),
 ('VA', 215),
 ('TN', 182),
 ('MA', 176),
 ('AZ', 174),
 ('SC', 156),
 ('AL', 153),
 ('IN', 150),
 ('MO', 141),
 ('KY', 126),
 ('MD', 125),
 ('WA', 122),
 ('MS', 105),
 ('LA', 104),
 ('CO', 98),
 ('OK', 93),
 ('WI', 91),
 ('CT', 86),
 ('MN', 84),
 ('UT', 83),
 ('AR', 66),
 ('NV', 60),
 ('KS', 58),
 ('OR', 57),
 ('WV', 46),
 ('IA', 38),
 ('NM', 38),
 ('NE', 35),
 ('RI', 32),
 ('ID', 29),
 ('NH', 29),
 ('DC', 25),
 ('HI', 25),
 ('DE', 20),
 ('AK', 18),
 ('ME', 18),
 ('MT', 18),
 ('WY', 13),
 ('SD', 12),
 ('ND', 9),
 ('VT', 8)]

In [25]:
c.close() # close the cursor

# Part 3- working with pony ORM

In [26]:
set_sql_debug(False)

## Pokemon dataset

In [27]:
!kaggle datasets download abcsds/pokemon -p ./datasets/
!unzip ./datasets/pokemon.zip -d ./datasets/pokemon/


  0%|          | 0.00/14.9k [00:00<?, ?B/s]
100%|##########| 14.9k/14.9k [00:00<00:00, 246kB/s]


Archive:  ./datasets/pokemon.zip
  inflating: ./datasets/pokemon/Pokemon.csv  


### First run (creating database)

In [28]:
db = Database()
db.bind(provider='sqlite', 
        filename=r'C:\Users\yossi\Jupyter notebooks\Data Science\Lab 1 (SQL)\datasets\pokemon\pokemon.pony.db', create_db=True)

class Trainer(db.Entity):
    name = Required(str)
    age = Required(int)
    pokemons_list = Set('Pokemon')
    
class Pokemon(db.Entity):
    name = Required(str) # cannot be None
    age  = Optional(int)
    attack_power = Required(int)
    defense_power = Required(int)
    trainer = Optional(Trainer)
db.generate_mapping(create_tables=True) # create tables

In [29]:
show(Trainer)

class Trainer(Entity):
    id = PrimaryKey(int, auto=True)
    name = Required(str)
    age = Required(int)
    pokemons_list = Set(Pokemon)


In [30]:
df = pd.read_csv('./datasets/pokemon/Pokemon.csv')
df

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [31]:
for idx, row in df.iterrows():
    Pokemon(name=row['Name'],
            age=random.randint(0,500),
            attack_power=row['Sp. Atk'], 
            defense_power=row['Sp. Def'])

In [32]:
commit()

In [33]:
ash_trainer = Trainer(name='Ash Ketchum', age=10)
pikachu = Pokemon(name='pikachu',age=6,attack_power=50, defense_power=50, trainer=ash_trainer)
charmander =  Pokemon(name='charmander',age=2,attack_power=60, defense_power=50,trainer=ash_trainer)
commit()

In [34]:
pikachu.trainer.age

10

In [35]:
list(pikachu.trainer.pokemons_list)

[Pokemon[802], Pokemon[801]]

In [36]:
list(pikachu.trainer.pokemons_list)[1].name

'pikachu'

In [37]:
pikachu.age

6

In [38]:
pikachu.age = 110
commit()

In [39]:
pikachu.age

110

### Second run 

In [40]:
db = Database()
db.bind(provider='sqlite', 
        filename=r'C:\Users\yossi\Jupyter notebooks\Data Science\Lab 1 (SQL)\datasets\pokemon\pokemon.pony.db')
# notice we don't use create_db=True

class Trainer(db.Entity):
    name = Required(str)
    age = Required(int)
    pokemons_list = Set('Pokemon')
    
class Pokemon(db.Entity):
    name = Required(str) # cannot be None
    age  = Optional(int)
    attack_power = Required(int)
    defense_power = Required(int)
    trainer = Optional(Trainer)
db.generate_mapping()
# notice we don't use create_tables=True

**Question 1:** Print the names of Pokemons which their attack is higher than 100 and their defence is higher than attack.  

In [41]:
[i.name for i in select(p for p in Pokemon if p.attack_power >100 and p.defense_power > p.attack_power )]

['Ho-oh',
 'MetagrossMega Metagross',
 'Latias',
 'LatiasMega Latias',
 'RotomHeat Rotom',
 'RotomWash Rotom',
 'RotomFrost Rotom',
 'RotomFan Rotom',
 'RotomMow Rotom',
 'Florges',
 'Sylveon',
 'Goodra']

**Question 2:** retrieve all the pokemon names that their attack is between 50 to 100.

In [42]:
[i.name for i in select(p for p in Pokemon if p.attack_power < 100 and p.attack_power > 50)]

['Bulbasaur',
 'Ivysaur',
 'Charmander',
 'Charmeleon',
 'Wartortle',
 'Blastoise',
 'Butterfree',
 'Pidgeot',
 'Fearow',
 'Arbok',
 'Raichu',
 'Nidorina',
 'Nidoqueen',
 'Nidorino',
 'Nidoking',
 'Clefairy',
 'Clefable',
 'Ninetales',
 'Wigglytuff',
 'Golbat',
 'Oddish',
 'Gloom',
 'Parasect',
 'Venomoth',
 'Persian',
 'Psyduck',
 'Golduck',
 'Primeape',
 'Growlithe',
 'Poliwrath',
 'Machamp',
 'Bellsprout',
 'Weepinbell',
 'Tentacruel',
 'Golem',
 'Ponyta',
 'Rapidash',
 'Magnemite',
 "Farfetch'd",
 'Dodrio',
 'Dewgong',
 'Muk',
 'Cloyster',
 'Hypno',
 'Voltorb',
 'Electrode',
 'Exeggcute',
 'Lickitung',
 'Koffing',
 'Weezing',
 'KangaskhanMega Kangaskhan',
 'Horsea',
 'Seadra',
 'Seaking',
 'Staryu',
 'Scyther',
 'Electabuzz',
 'Pinsir',
 'PinsirMega Pinsir',
 'Gyarados',
 'GyaradosMega Gyarados',
 'Lapras',
 'Flareon',
 'Porygon',
 'Omanyte',
 'Kabuto',
 'Kabutops',
 'Aerodactyl',
 'AerodactylMega Aerodactyl',
 'Snorlax',
 'Articuno',
 'Dragonair',
 'Bayleef',
 'Meganium',
 'Cyndaq

**Question 3**: find the pokemon with the highest attack power:

In [43]:
max_attack = max([i.attack_power for i in select(p for p in Pokemon)])
max_attack

194

In [44]:
[i.name for i in select(p for p in Pokemon if p.attack_power == max_attack)]

['MewtwoMega Mewtwo Y']

## Kickstarter projects dataset

In [45]:
os.mkdir('./datasets/kick-starter/')
url = 'https://github.com/maniknarang/kickstarter-data-analysis/raw/master/ks-projects-201801.csv'
filename = './datasets/kick-starter/ks_dataset.txt'
urllib.request.urlretrieve(url, filename)

('./datasets/kick-starter/ks_dataset.txt',
 <http.client.HTTPMessage at 0x1ea7fd53700>)

##### Google colab:

!wget https://github.com/maniknarang/kickstarter-data-analysis/raw/master/ks-projects-201801.csv

In [46]:
df = pd.read_csv("./datasets/kick-starter/ks_dataset.txt", encoding="utf-8", delimiter=',')
df

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378656,999976400,ChknTruk Nationwide Charity Drive 2014 (Canceled),Documentary,Film & Video,USD,2014-10-17,50000.0,2014-09-17 02:35:30,25.0,canceled,1,US,25.0,25.0,50000.00
378657,999977640,The Tribe,Narrative Film,Film & Video,USD,2011-07-19,1500.0,2011-06-22 03:35:14,155.0,failed,5,US,155.0,155.0,1500.00
378658,999986353,Walls of Remedy- New lesbian Romantic Comedy f...,Narrative Film,Film & Video,USD,2010-08-16,15000.0,2010-07-01 19:40:30,20.0,failed,1,US,20.0,20.0,15000.00
378659,999987933,BioDefense Education Kit,Technology,Technology,USD,2016-02-13,15000.0,2016-01-13 18:13:53,200.0,failed,6,US,200.0,200.0,15000.00


**Question 1:** Use Pony for creating a new database.

The database should include a Project table with a name, category and country as strings, whreas pledged and goal as floats.
Fill the table using the CSV file.


In [47]:
db = Database()
db.bind(provider='sqlite', filename=r'C:\Users\yossi\Jupyter notebooks\Data Science\Lab 1 (SQL)\datasets\kick-starter\projects.pony', create_db=True) # notice we don't use create_db=True here

In [48]:
class Project(db.Entity):
    name = Required(str)
    category = Required(str)
    country = Required(str)
    pledged = Required(float)
    goal = Required(float)
    
db.generate_mapping(create_tables=True)

In [49]:
for idx, row in df.iterrows():
    Project(name=str(row['name']),
            category = row['category'],
            country = row['country'],
            pledged = row['pledged'],
            goal = row['usd_goal_real'])

In [50]:
commit()

**Question 2**: Write a Pony code which retrieve the project's names of those which their contry is "US".

In [51]:
[i.name for i in select(p for p in Project if p.country == 'US')]

['Greeting From Earth: ZGAC Arts Capsule For ET',
 'Where is Hank?',
 'ToshiCapital Rekordz Needs Help to Complete Album',
 'Community Film Project: The Art of Neighborhood Filmmaking',
 'Monarch Espresso Bar',
 'Support Solar Roasted Coffee & Green Energy!  SolarCoffee.co',
 'Chaser Strips. Our Strips make Shots their B*tch!',
 'SPIN - Premium Retractable In-Ear Headphones with Mic',
 'STUDIO IN THE SKY - A Documentary Feature Film (Canceled)',
 'Lisa Lim New CD!',
 'The Cottage Market',
 'G-Spot Place for Gamers to connect with eachother & go pro!',
 'Survival Rings',
 'The Beard',
 'Notes From London: Above & Below',
 "Mike Corey's Darkness & Light Album",
 'Boco Tea',
 'CMUK. Shoes: Take on Life Feet First.',
 'Alice in Wonderland in G Minor',
 'The Book Zoo - A Mini-Comic',
 'Matt Cavenaugh & Jenny Powers make their 1st album!',
 'My Moon - Animated Short Film',
 'Ledr workbook: one tough journal!',
 'BB130A',
 'Chris Eger Band - New Nashville Record!',
 'Squatch Watchers',
 'Arro

## Further Readings and Practice


*   [SQLZoo](https://sqlzoo.net)
*   [The SQL Murder Mystery](https://mystery.knightlab.com)
*   [W3 School SQLL Tutorial](https://www.w3schools.com/sql/)
*   [PonyORM](https://ponyorm.org)
*   [Kaggle - Intro to SQL](https://www.kaggle.com/learn/intro-to-sql)
*   [Getting Started with SQL and BigQuery notebook](https://www.kaggle.com/dansbecker/getting-started-with-sql-and-bigquery)